## Customer Segmentation Project

#### Project Overview
Objective of this Project is to perform Customer Segmentation Analysis based on the transaction history of an e-Commerce business from 01/12/2010 to 09/12/2011 extracted from AWS Redshift

#### Project is divided into 3 sections:
1. Exploring and Cleaning Data
2. Data Analysis and Visualization
3. Cohort, RFM and KMean Analysis

* There is an extra section which utilises SQL to query information to answer business questions

### 1. Exploring and cleaning data 

In this section, the dataset are explored and prepared for further customer segmentation analysis tasks

In [1]:
# Import the required libraries

import psycopg2
import pandas as pd
import numpy as np

# using this library for reading the password
from dotenv import load_dotenv
import os

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Import environment variables from .env file

load_dotenv()

dbname = os.getenv("dbname")
host = os.getenv("host")
port = os.getenv("port")
user = os.getenv("user")
password = os.getenv("password")

In [3]:
# Connect to AWS Redshift

connect = psycopg2.connect(dbname=dbname, host=host, port=port, user=user, password=password)
cursor = connect.cursor()

In [4]:
# Query the informations in the tables(stock_description)

query = """
SELECT * 
FROM bootcamp.stock_description;
"""

stock_description = pd.read_sql(query, connect)
stock_description

,stock_code,description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER
3,10123C,HEARTS WRAPPING TAPE
4,10124A,SPOTS ON RED BOOKCOVER TAPE
...,...,...
3947,DCGSSGIRL,GIRLS PARTY BAG
3948,DOT,DOTCOM POSTAGE
3949,PADS,PADS TO MATCH ALL CUSHIONS
3950,POST,POSTAGE


In [5]:
stock_description.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3952 entries, 0 to 3951
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   stock_code   3952 non-null   object
 1   description  3952 non-null   object
dtypes: object(2)
memory usage: 61.9+ KB


In [6]:
stock_description.describe()

,stock_code,description
count,3952,3952
unique,3905,3785
top,22600,?
freq,2,47


In [7]:
stock_description.isna().sum()

stock_code     0
description    0
dtype: int64

In [8]:
stock_description.duplicated().sum()

0

In [9]:
stock_description['description'].value_counts()

description
?                                     47
METAL SIGN,CUPCAKE SINGLE HOOK         6
CINAMMON SET OF 9 T-LIGHTS             2
COLUMBIAN CANDLE RECTANGLE             2
3 WHITE CHOC MORRIS BOXED CANDLES      2
                                      ..
MAGNETS PACK OF 4 SWALLOWS             1
MAGNETS PACK OF 4 CHILDHOOD MEMORY     1
MAGNETS PACK OF 4 HOME SWEET HOME      1
MAGNETS PACK OF 4 VINTAGE COLLAGE      1
SAMPLES                                1
Name: count, Length: 3785, dtype: int64

In [10]:
query = """
SELECT *
FROM bootcamp.stock_description sd
WHERE description = '?';
"""

pd.read_sql(query, connect)

,stock_code,description
0,16020C,?
1,16207B,?
2,21145,?
3,21232,?
4,21368,?
5,21427,?
6,21446,?
7,21591,?
8,21877,?
9,22077,?


Upon investigation, all the stocks with '?' description in the stock_description table have a corresponding description with the same stock_code, so this data will be deleted from the table 

In [11]:
# Query the informations in the tables(online_transactions)

query = """
SELECT * 
FROM bootcamp.online_transactions;
"""

online_transactions = pd.read_sql(query, connect)
online_transactions

,invoice,stock_code,quantity,invoice_date,price,customer_id,country
0,536370,21791,24,2010-12-01 08:45:00,1.25,u12583,France
1,536373,82494L,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom
2,536378,21929,10,2010-12-01 09:37:00,1.95,u14688,United Kingdom
3,536381,37444A,1,2010-12-01 09:41:00,2.95,u15311,United Kingdom
4,536381,15056BL,2,2010-12-01 09:41:00,5.95,u15311,United Kingdom
...,...,...,...,...,...,...,...
541905,581580,22698,1,2011-12-09 12:20:00,2.95,u12748,United Kingdom
541906,581584,20832,72,2011-12-09 12:25:00,0.72,u13777,United Kingdom
541907,581585,22178,12,2011-12-09 12:31:00,1.95,u15804,United Kingdom
541908,581585,84692,25,2011-12-09 12:31:00,0.42,u15804,United Kingdom


In [12]:
# Check for online_transactions information

online_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   invoice       541910 non-null  object 
 1   stock_code    541910 non-null  object 
 2   quantity      541910 non-null  int64  
 3   invoice_date  541910 non-null  object 
 4   price         541910 non-null  float64
 5   customer_id   541910 non-null  object 
 6   country       541910 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 28.9+ MB


In [13]:
online_transactions.isna().sum()

invoice         0
stock_code      0
quantity        0
invoice_date    0
price           0
customer_id     0
country         0
dtype: int64

In [14]:
# total_duplicated rows
online_transactions.duplicated().sum()

5270

There is a total of 5270 duplicated rows that will be removed from the dataset

In [15]:
online_transactions['customer_id'].value_counts()

customer_id
          135080
u17841      7983
u14911      5903
u14096      5128
u12748      4642
           ...  
u15369         1
u17448         1
u16093         1
u14119         1
u18174         1
Name: count, Length: 4373, dtype: int64

There is a total of 135080 invoices with ' ' customer_id that will be removed from the dataset

In [16]:
online_transactions['invoice'].value_counts()

invoice
573585     1114
581219      749
581492      731
580729      721
558475      705
           ... 
554828        1
C554687       1
C540796       1
554647        1
C581384       1
Name: count, Length: 25900, dtype: int64

In [17]:
online_transactions['country'].value_counts()

country
United Kingdom          495478
Germany                   9495
France                    8558
EIRE                      8196
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               2002
Portugal                  1519
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Unspecified                446
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
Israel                     297
USA                        291
Hong Kong                  288
Singapore                  229
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United Arab Emirates        68
European Community          61
RSA                         58


In [18]:
online_transactions.describe()

,quantity,price
count,541910.000000,541910.000000
mean,9.552234,4.611138
std,218.080957,96.759765
min,-80995.000000,-11062.060000
25%,1.000000,1.250000
50%,3.000000,2.080000
75%,10.000000,4.130000
max,80995.000000,38970.000000


In [19]:
alpha_stock_code = ['BANK CHARGES', 'POST', 'D', 'M', 'CRUK', 'C2', 'DOT', 'PADS']
online_transactions[online_transactions['stock_code'].isin(alpha_stock_code)]

,invoice,stock_code,quantity,invoice_date,price,customer_id,country
227,536403,POST,1,2010-12-01 11:27:00,15.0,u12791,Netherlands
582,536852,POST,1,2010-12-03 09:51:00,18.0,u12686,France
672,536967,POST,1,2010-12-03 12:57:00,18.0,u126,Germany
724,537026,POST,2,2010-12-03 16:35:00,18.0,u12395,Belgium
735,536370,POST,3,2010-12-01 08:45:00,18.0,u12583,France
...,...,...,...,...,...,...,...
541427,581578,POST,3,2011-12-09 12:16:00,18.0,u12713,Germany
541451,581171,POST,2,2011-12-07 15:02:00,18.0,u12615,France
541466,581179,POST,1,2011-12-07 15:43:00,240.0,u12471,Germany
541613,581266,POST,5,2011-12-08 11:25:00,18.0,u12621,Germany


'BANK CHARGES', 'POST', 'D', 'M', 'CRUK', 'C2', 'DOT', 'PADS' stock_codes are not the actual products shipped to the customers but for postage and other charges. These invoices will be removed from the dataset

Bank Charges = Bank Charges, Post = Postage, D = Discount, M = Manual, CRUK = CRUK Commission, C2 = Carriage, DOT = Dotcom postage, PADS = Pads to Match all Cushions

In [20]:
# Investigate invoice numbers

online_transactions['invoice'].str[0].value_counts()

invoice
5    532619
C      9288
A         3
Name: count, dtype: int64

In [21]:
# Preview records where invoice number starts with C

online_transactions[online_transactions['invoice'].str[0] == 'C'].sort_values(by = 'quantity')

,invoice,stock_code,quantity,invoice_date,price,customer_id,country
531556,C581484,23843,-80995,2011-12-09 09:27:00,2.08,u16446,United Kingdom
173528,C541433,23166,-74215,2011-01-18 10:17:00,1.04,u12346,United Kingdom
4924,C536757,84347,-9360,2010-12-02 14:23:00,0.03,u15838,United Kingdom
103104,C550456,21108,-3114,2011-04-18 13:08:00,2.10,u15749,United Kingdom
92191,C550456,21175,-2000,2011-04-18 13:08:00,1.85,u15749,United Kingdom
...,...,...,...,...,...,...,...
357262,C574712,21539,-1,2011-11-06 14:25:00,4.95,u14502,United Kingdom
218843,C567627,22383,-1,2011-09-21 13:01:00,1.65,u17511,United Kingdom
123994,C553033,POST,-1,2011-05-12 19:50:00,3.50,u13883,United Kingdom
355880,C560045,84692,-1,2011-07-14 14:51:00,0.42,u16779,United Kingdom


Invoices that starts with 'C' are cancellation or return orders that all has negative values

In [22]:
# Find the cancellation pairs: original invoice and corresponding cancelled invoice

online_transactions.groupby([online_transactions.customer_id, 
                             online_transactions.stock_code, 
                             online_transactions.quantity.abs()])\
                   .filter(lambda x: (len(x.quantity.abs()) % 2 == 0) and (x.quantity.sum() == 0))\
                   .sort_values(by=['customer_id', 'stock_code'])

,invoice,stock_code,quantity,invoice_date,price,customer_id,country
49390,543899,15056N,12,2011-02-14 12:11:00,5.95,,EIRE
135708,C557663,15056N,-12,2011-06-21 17:59:00,5.95,,EIRE
74979,543899,20658,12,2011-02-14 12:11:00,1.25,,EIRE
160892,C557663,20658,-12,2011-06-21 17:59:00,1.25,,EIRE
154714,560772,20666,5,2011-07-20 16:12:00,0.83,,United Kingdom
...,...,...,...,...,...,...,...
379047,C577832,23245,-4,2011-11-22 10:18:00,4.95,u18274,United Kingdom
360941,575485,84509A,4,2011-11-09 17:03:00,3.75,u18274,United Kingdom
497501,C577832,84509A,-4,2011-11-22 10:18:00,3.75,u18274,United Kingdom
337348,C577832,84988,-12,2011-11-22 10:18:00,1.45,u18274,United Kingdom


The transactions with its direct corresponding cancellation/return orders will be removed from the dataset

In [23]:
# Investigate price with 0.00 values

online_transactions[online_transactions['price'] == 0]

,invoice,stock_code,quantity,invoice_date,price,customer_id,country
341,536589,21777,-10,2010-12-01 16:50:00,0.0,,United Kingdom
716,537002,21651,70,2010-12-03 15:33:00,0.0,,United Kingdom
717,537006,21648,-130,2010-12-03 15:36:00,0.0,,United Kingdom
718,537014,21040,-20,2010-12-03 15:40:00,0.0,,United Kingdom
1238,536547,37509,1,2010-12-01 14:33:00,0.0,,United Kingdom
...,...,...,...,...,...,...,...
538379,579683,21868,-28,2011-11-30 13:35:00,0.0,,United Kingdom
538608,579733,84581,-41,2011-11-30 14:24:00,0.0,,United Kingdom
538629,579755,84912B,-14,2011-11-30 14:54:00,0.0,,United Kingdom
538967,580366,22923,6,2011-12-02 16:38:00,0.0,,United Kingdom


In [24]:
query = """
SELECT ot.invoice,
       ot.stock_code,
       CASE WHEN s.description is null then 'Unknown'
            ELSE s.description END as description,
       ot.price,
       ot.quantity,
       /* add variable of total_order_value*/
       ot.price*ot.quantity AS total_order_value,
       CAST(invoice_date as DateTime) AS invoice_date,
       ot.customer_id,
       ot.country
FROM bootcamp.online_transactions ot
LEFT JOIN (SELECT * /*subqueries to not include '?' description*/
           FROM bootcamp.stock_description sd 
           WHERE description <> '?') AS s ON ot.stock_code = s.stock_code
           WHERE ot.customer_id <> '' AND
           ot.stock_code NOT IN ('BANK CHARGES', 'POST', 'D', 'M', 'CRUK', 'C2', 'DOT', 'PADS')
           """

online_trans_w_desc = pd.read_sql(query, connect)
online_trans_w_desc.shape

(404909, 9)

In [25]:
online_trans_w_desc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404909 entries, 0 to 404908
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   invoice            404909 non-null  object        
 1   stock_code         404909 non-null  object        
 2   description        404909 non-null  object        
 3   price              404909 non-null  float64       
 4   quantity           404909 non-null  int64         
 5   total_order_value  404909 non-null  float64       
 6   invoice_date       404909 non-null  datetime64[ns]
 7   customer_id        404909 non-null  object        
 8   country            404909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 27.8+ MB


In [26]:
# Removes duplicate lines from table

online_trans_w_desc = online_trans_w_desc.drop_duplicates(keep = 'first')
online_trans_w_desc.shape

(399687, 9)

In [27]:
# Drop records with pairs of original invoice and matching cancelled invoice

idx = online_trans_w_desc.groupby([online_trans_w_desc.customer_id, online_trans_w_desc.stock_code, online_trans_w_desc.quantity.abs()])\
            .filter(lambda x: (len(x.quantity.abs()) % 2 == 0) and (x.quantity.sum() == 0)).index

online_trans_w_desc = online_trans_w_desc.drop(idx)

In [28]:
online_trans_w_desc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 395661 entries, 0 to 404908
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   invoice            395661 non-null  object        
 1   stock_code         395661 non-null  object        
 2   description        395661 non-null  object        
 3   price              395661 non-null  float64       
 4   quantity           395661 non-null  int64         
 5   total_order_value  395661 non-null  float64       
 6   invoice_date       395661 non-null  datetime64[ns]
 7   customer_id        395661 non-null  object        
 8   country            395661 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 30.2+ MB


In [29]:
# Load the preprocessed dataframe to a .pkl file in local folder

online_trans_w_desc.to_pickle('../data/online_trans_cleaned.pkl')

#### Tasks performed in this section: 
1. extracts bootcamp.online_transactions table and bootcamp.stock_description table from AWS Redshift
2. joins the description field from stock_description to the online_transactions table
3. replaces missing descriptions with 'Unknown'
4. creates a variable called 'total_order_value'(quantity * price)
5. converts the invoice date to a datetime field
6. removes invoices where customer id is ' '
7. removes invoices where stock code is in 'BANK CHARGES', 'POST', 'D', 'M', 'CRUK', 'C2', 'DOT', 'PADS'
8. removes duplicates transactions
9. removes cancelled/returned orders that have a direct match

* The data obtained from the tasks performed in this section are saved in a local folder for further analysis